In [44]:
from typing import Literal

class Computer:
    def __init__(self, a: int, instructions: list[int], early_stop=False, verbose=False):
        self.a = a
        self.instructions = instructions
        self.b = 0
        self.c = 0
        self.instruction_pointer = 0
        self.output = []
        self.early_stop = early_stop
        self.verbose = verbose   
        
    def get_output(self) -> str:
        return ','.join([str(i) for i in self.output])
        
    def __get_combo_operand(self, operand: int):
        if operand < 4:
            return operand
        if operand == 4:
            return self.a
        if operand == 5:
            return self.b
        if operand == 6:
            return self.c
        raise ValueError(f"Invalid combo operand {operand}")
        
    def dv(self, operand: int, register: Literal['a', 'b', 'c']) -> None:
        temp = self.a // (2**self.__get_combo_operand(operand))
        setattr(self, register, temp)
        self.verbose and print(f"Setting {register} to A // {2**self.__get_combo_operand(operand):o})")
        self.verbose and print(f"A = {self.a: <15o}, B = {self.b: <15o}, C = {self.c: <15o}")
        
    def bxl(self, operand: int) -> None:
        self.b ^= operand
        self.verbose and print(f"XORing B with {operand:o}")
        self.verbose and print(f"A = {self.a: <15o}, B = {self.b: <15o}, C = {self.c: <15o}")
    
    def bst(self, operand: int) -> None:
        self.b = self.__get_combo_operand(operand) % 8
        self.verbose and print(f"Setting B to combo({operand}) % 8")
        self.verbose and print(f"A = {self.a: <15o}, B = {self.b: <15o}, C = {self.c: <15o}")
        
    def jnz(self, operand: int) -> None:
        if self.a == 0:
            return
        
        self.instruction_pointer = operand - 2
        self.verbose and print("Repeat!")
        
    def bxc(self, operand: int) -> None:
        self.b ^= self.c
        self.verbose and print(f"Setting B to B XOR C")
        self.verbose and print(f"A = {self.a: <15o}, B = {self.b: <15o}, C = {self.c: <15o}")
        
    def out(self, operand: int) -> None:
        temp = self.__get_combo_operand(operand) % 8
        self.verbose and print(f"Outputting combo({operand}) % 8 = {temp:o}")
        self.verbose and print(f"A = {self.a: <15o}, B = {self.b: <15o}, C = {self.c: <15o}")
        self.output.append(temp)
        return temp == self.instructions[len(self.output) - 1]
        
    def adv(self, operand: int) -> None:
        return self.dv(operand, 'a')

    def bdv(self, operand: int) -> None:
        return self.dv(operand, 'b')
    
    def cdv(self, operand: int) -> None:
        return self.dv(operand, 'c')
    
    def reset(self, a: int) -> None:
        self.a = a
        self.b = 0
        self.c = 0
        self.instruction_pointer = 0
        self.output = []
    
    def run(self) -> bool:
        while self.instruction_pointer < len(self.instructions):
            instruction = self.instructions[self.instruction_pointer]
            operand = self.instructions[self.instruction_pointer + 1]
            if instruction == 0:
                self.adv(operand)
            elif instruction == 1:
                self.bxl(operand)
            elif instruction == 2:
                self.bst(operand)
            elif instruction == 3:
                self.jnz(operand)
            elif instruction == 4:
                self.bxc(operand)
            elif instruction == 5:
                if not self.out(operand) and self.early_stop:
                    return False
            elif instruction == 6:
                self.bdv(operand)
            elif instruction == 7:
                self.cdv(operand)
                
            self.instruction_pointer += 2
            
        return (len(self.output) == len(self.instructions)) and all(x == y for x, y in zip(self.output, self.instructions))
    
with open('input.txt') as f:
    lines = f.readlines()
    
reg_a = int(lines[0].strip()[11:])
instructions = [int(i) for i in lines[4].strip()[8:].split(',')]

# Part 1

In [45]:
computer = Computer(reg_a, instructions, verbose=True)
computer.run()
computer.get_output()

Setting B to combo(4) % 8
A = 347103303      , B = 3              , C = 0              
XORing B with 5
A = 347103303      , B = 6              , C = 0              
Setting c to A // 100)
A = 347103303      , B = 6              , C = 3471033        
XORing B with 6
A = 347103303      , B = 0              , C = 3471033        
Setting B to B XOR C
A = 347103303      , B = 3471033        , C = 3471033        
Outputting combo(5) % 8 = 3
A = 347103303      , B = 3471033        , C = 3471033        
Setting a to A // 10)
A = 34710330       , B = 3471033        , C = 3471033        
Repeat!
Setting B to combo(4) % 8
A = 34710330       , B = 0              , C = 3471033        
XORing B with 5
A = 34710330       , B = 5              , C = 3471033        
Setting c to A // 40)
A = 34710330       , B = 5              , C = 716206         
XORing B with 6
A = 34710330       , B = 3              , C = 716206         
Setting B to B XOR C
A = 34710330       , B = 716205         , C = 716206     

'3,5,0,1,5,1,5,1,0'

# Part 2

My Program is as follows:

```python	
B = A & 7       # Get the last 3 bits of A
B = B ^ 0b101   # Invert bit 2 and 0
C = A >> B      # Shift A right by B bits
B = b ^ 0b110   # Invert bit 1 and 2
B = B ^ C       # B XOR C
OUT: B & 7      # Output the last 3 bits of B
A = A >> 3      # Shift A right by 3 bits
REPEAT          # Repeat
```

```python	
B1 = A & 7       # Get the last 3 bits of A
B2 = B1 ^ b101   # Invert bit 2 and 0
C  = A1 >> B2    # Shift A right by B bits
B3 = B2 ^ b110   # Invert bit 1 and 2
B4 = B3 ^ C      # B XOR C
OUT: B4 & 7      # Output the last 3 bits of B
A = A >> 3       # Shift A right by 3 bits
REPEAT           # Repeat
```



In [46]:
octets = [0 for _ in range(15)]
for i, instruction in enumerate(instructions[::-1]):
    b = instruction & 0b111
    c = octets[i+b//8]
        

IndexError: list index out of range

In [43]:
import math

for i in range(8):
    print(i, math.log(2**i, 8))

0 0.0
1 0.33333333333333337
2 0.6666666666666667
3 1.0
4 1.3333333333333335
5 1.6666666666666667
6 2.0
7 2.3333333333333335
